In [2]:
push!(LOAD_PATH, "..")

4-element Array{ByteString,1}:
 "/Applications/Julia-0.4.1.app/Contents/Resources/julia/local/share/julia/site/v0.4"
 "/Applications/Julia-0.4.1.app/Contents/Resources/julia/share/julia/site/v0.4"      
 "/Users/rdeits/6.338/hw2"                                                           
 ".."                                                                                

In [26]:
reload("Ev3")
import Ev3

In [54]:
motor = Ev3.Motor("outA", "192.168.1.27")

Ev3.Motor(Ev3.RemoteNode("/sys/class/tacho-motor/motor0","192.168.1.27",ZMQ.Socket(Ptr{Void} @0x00007fc7c217a000,Base._FDWatcher(Ptr{Void} @0x00007fc7c16b2710,(1,0),Condition(Any[]),(false,false)))),Set(SubString{ASCIIString}["run-timed","stop","run-direct","run-to-rel-pos","reset","run-forever","run-to-abs-pos"]))

In [55]:
Ev3.commands(motor)

Set(SubString{ASCIIString}["run-timed","stop","run-direct","run-to-rel-pos","reset","run-forever","run-to-abs-pos"])

In [56]:
write(motor, "speed_regulation", "on")
Ev3.speed_sp(motor, 100)
write(motor, "command", "run-forever")

""

In [57]:
Ev3.speed_sp(motor, 170)
Ev3.speed_sp(motor)

"170"

In [58]:
Ev3.stop(motor, "brake")

""

In [18]:
us = Ev3.Sensor("in1", "192.168.1.27")

Ev3.Sensor(Ev3.RemoteNode("/sys/class/lego-sensor/sensor1","192.168.1.27",ZMQ.Socket(Ptr{Void} @0x00007fd9bc807e00,Base._FDWatcher(Ptr{Void} @0x00007fd9bc5fbf60,(1,0),Condition(Any[]),(false,false)))))

In [5]:
Ev3.value0(us)

321

In [27]:
gyro = Ev3.Sensor("in4", "192.168.1.27")

Ev3.Sensor(Ev3.RemoteNode("/sys/class/lego-sensor/sensor3","192.168.1.27",ZMQ.Socket(Ptr{Void} @0x00007fd9bb4ee800,Base._FDWatcher(Ptr{Void} @0x00007fd9bf2df8e0,(1,0),Condition(Any[]),(false,false)))))

In [31]:
@time for j = 1:10; Ev3.value0(gyro); end

  1.213552 seconds (279 allocations: 13.203 KB)


In [24]:
module Mapping

import Ev3

type Sides{T}
    right::T
    left::T
end 

type Odometer
    motor::Ev3.Motor
    ticks_per_revolution::Real
    total_revolutions::Real
    last_position::Real
end

function Odometer(motor::Ev3.Motor)
    ticks_per_revolution = read(motor, "counts_per_rot", Int)
    current_position = read(motor, "position", Int)
    Odometer(motor, ticks_per_revolution, 0.0, current_position)
end

function update(odo::Odometer)
    current_position = get_position(odo.motor)
    delta = current_position - odo.last_position
    if abs(delta) > (odo.ticks_per_revolution / 2.0)
        error("wraparound not implemented")
    end
    odo.last_position = current_position
    total_revolutions += delta / odo.ticks_per_revolution
end

type StateEstimator
    odos::Sides{Odometer}
    gyro::Sensor
    saved_orientation
end

function update(se::StateEstimator)
    update(odos.right)
    update(odos.left)
    

type MappingRobot
    motors::Sides{Ev3.Motor}
    odos::Sides{Odometer}
    distance_sensor::Ev3.Sensor
end

function MappingRobot(hostname)
    motors = Sides(Ev3.Motor("outA", hostname), Ev3.Motor("outB", hostname))
    odos = Sides(Odometer(motors.right), Odometer(motors.left))
    sensor = Ev3.Sensor("in1", hostname)
    MappingRobot(motors, odos, sensor)
end

type RobotState
    translation
    yaw
end

type Map
    points

end

end

Mapping